# Starling Task across subject behavioral analysis combined
Let's start the analysis based on Rhiannon's list [here](https://uofutah-my.sharepoint.com/:w:/g/personal/u1363968_umail_utah_edu/ESn4E7plikFIs1ZyLHy5YaUBZfn_td7fv2yCh6I5HsWL2g?e=MxDJfG&CID=0428038a-a81f-6b6f-5c00-c8f4ada097eb).

By: Niloufar Shahdoust

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('svg')
matplotlib.rcParams['svg.fonttype'] = 'none'
matplotlib.rcParams['font.weight'] = 'bold'
from matplotlib.patches import Patch
import os
import seaborn as sns 
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FixedLocator
from scipy.stats import permutation_test
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from scipy.optimize import curve_fit
import os



# ****************************************************************************

## reading all subjects data

In [38]:
folder_path = 'data_risk_added'
folder_path_epileptic = 'data_risk_added_epileptic'

output_folder = '4_output_healthy_epileptic_combined'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

if not os.path.exists(folder_path_epileptic):
    os.makedirs(folder_path_epileptic)

dataframes = []
dataframes_epileptic = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_excel(file_path)
        dataframes.append(df)



for file_name in os.listdir(folder_path_epileptic):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path_epileptic, file_name)
        df = pd.read_csv(file_path)
        dataframes_epileptic.append(df)
        

In [39]:
dataframes_epileptic[0]

,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,is_within_IQR,risk
0,2390,uniform,756,win,2,7,3186,10.5,0,response,arrowdown,1,0,0,0.125
1,1262,uniform,874,win,9,4,1040,11,1,response,arrowup,1,0,0,0.000
2,1547,uniform,791,win,5,6,204,11.5,2,response,arrowdown,1,0,1,0.500
3,1627,uniform,828,win,9,1,872,12,3,response,arrowup,1,0,0,0.000
4,459,uniform,894,win,6,4,790,12.5,4,response,arrowup,1,0,0,0.375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,514,high,847,win,8,7,429,85,134,response,arrowup,4,0,1,0.243
268,179,high,902,win,4,8,211,85.5,115,response,arrowdown,4,0,1,0.146
269,419,low,815,lose,8,9,353,85,81,response,arrowup,4,0,1,0.023
270,731,low,941,lose,3,1,212,84.5,59,response,arrowdown,4,0,1,0.447


# ****************************************************************************

In [40]:
for df in dataframes:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix

    

for df in dataframes_epileptic:
    df['block_type'] = None

    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix

In [41]:
for df in dataframes:
    df.drop(df[df['arrowRT'] == 'na'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

for df in dataframes_epileptic:
    df.drop(df[df['arrowRT'] == 'na'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)


## number of participants

In [42]:
n_participant = len(dataframes)
print(f"there are {n_participant} healthy participants.")


n_participant_epileptic = len(dataframes_epileptic)
print(f"there are {n_participant_epileptic}  epileptic participants.")

there are 38 healthy participants.
there are 9  epileptic participants.


### visualization prerequisites:
this order is very important in adding all the analysis block labels!

In [43]:
x_labels = ['uniform','low', 'high']
distributions_to_show = ['uniform','low', 'high']
colors = ['#808080',  '#ff7f0e', '#2ca02c']

# ****************************************************************************

<h2 style="color: green;">(0) looking at totalReward based on trials:</h2>

In [44]:
plt.figure(figsize=(3, 3))

# --- Combine Healthy and Epileptic ---
participant_totalReward_list = []

# Add healthy participants
for df in dataframes:
    rewards = df[df['totalReward'] != "na"]['totalReward'].astype(float).tolist()
    participant_totalReward_list.append(rewards)
    plt.plot(rewards, color='lightgray', alpha=0.8, linewidth=0.2)

# Add epileptic participants
for df in dataframes_epileptic:
    rewards = df[df['totalReward'] != "na"]['totalReward'].astype(float).tolist()
    participant_totalReward_list.append(rewards)
    plt.plot(rewards, color='lightgray', alpha=0.8, linewidth=0.2)

# Mean curve across all participants
mean_total_reward = np.mean(participant_totalReward_list, axis=0)
plt.plot(mean_total_reward, color='black', linewidth=0.5, label='mean total reward')

# --- Permutation Test Between Groups ---
healthy_means = np.mean(participant_totalReward_list[:len(dataframes)], axis=1)
epileptic_means = np.mean(participant_totalReward_list[len(dataframes):], axis=1)

combined = np.concatenate([healthy_means, epileptic_means])
n_healthy = len(healthy_means)
n_permutations = 10000
permuted_diffs = []

np.random.seed(0)
for _ in range(n_permutations):
    permuted = np.random.permutation(combined)
    group1 = permuted[:n_healthy]
    group2 = permuted[n_healthy:]
    diff = np.mean(group1) - np.mean(group2)
    permuted_diffs.append(diff)

permuted_diffs = np.array(permuted_diffs)

# --- Final Plot Settings ---
plt.xlabel("trial", fontsize=12, fontweight='bold')
plt.ylabel("total reward", fontsize=12, fontweight='bold')
plt.title("total reward across trials", fontsize=14, fontweight='bold')

# Bold font for legend
plt.legend(frameon=False, fontsize=12, loc='upper left', prop={'weight': 'bold'})

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Bold tick labels
for tick in ax.get_xticklabels():
    tick.set_fontweight('bold')
for tick in ax.get_yticklabels():
    tick.set_fontweight('bold')

ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)


plt.tight_layout()
plt.savefig(os.path.join(output_folder, "total_reward_combined.pdf"), format="pdf", dpi=300, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "total_reward_combined.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_25696\3247608303.py:66: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


<h2 style="color: green;">(1) Accuracy.</h2>

In [45]:
distributions_to_show = ['uniform', 'low', 'high']
colors = ['#808080', '#ff7f0e', '#2ca02c']

combined_vals_all = {dist: [] for dist in distributions_to_show}

# Combine healthy and epileptic, fix and mix
for df in dataframes + dataframes_epileptic:
    df['is_win'] = df['outcome'].apply(lambda x: 1 if x == 'win' else 0)
    for dist in distributions_to_show:
        acc = df[df['distribution'] == dist]['is_win'].mean()
        combined_vals_all[dist].append(acc)

fig, ax = plt.subplots(figsize=(2.5, 2.5))

for i, dist in enumerate(distributions_to_show):
    ax.boxplot(
        combined_vals_all[dist],
        positions=[i],
        widths=0.3,
        patch_artist=True,
        boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.3),
        medianprops=dict(color='black', linewidth=0.3),
        whiskerprops=dict(color='black', linewidth=0.3),
        capprops=dict(color='black', linewidth=0.3),
        showfliers=False
    )

    # Scatter
    jitter_x = np.random.normal(i, 0.05, size=len(combined_vals_all[dist]))
    jitter_y = np.array(combined_vals_all[dist]) + np.random.normal(0, 0.005, size=len(combined_vals_all[dist]))
    ax.scatter(jitter_x, jitter_y, s=3, color=colors[i], alpha=0.6, edgecolors='none')

    # Annotation
    ax.text(i, 0.45, dist, ha='center', va='top', fontsize=10, fontweight='bold')

ax.set_xticks([])
ax.set_title("accuracy", fontsize=14, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_ylabel('accuracy', fontsize=12, fontweight='bold')
ax.set_ylim(0, 1)


plt.tight_layout()
plt.savefig(os.path.join(output_folder, "combined_accuracy.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "combined_accuracy.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_25696\1074662736.py:47: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


<h2 style="color: green;"> space RT</h2>

In [46]:
# --- shared settings ---
distributions_to_show = ['uniform', 'low', 'high']
colors = ['#808080', '#ff7f0e', '#2ca02c']
bar_width = 0.35
fig, ax = plt.subplots(figsize=(2.5,2.5))
x = np.arange(len(distributions_to_show))

# --- combine and filter participants ---
spaceRT_fixed_all = []

def process_df(df_list):
    for df in df_list:
        df = df[df['spaceRT'] <= 1500]  # remove high RTs
        df_fix = df[(df['block_type'] == 'fix') & (df['is_within_IQR'] == 1)]
        spaceRT_fixed_all.append(df_fix.groupby('distribution')['spaceRT'].mean())

process_df(dataframes)
process_df(dataframes_epileptic)

# box/points – only FIX
for i, dist in enumerate(distributions_to_show):
    vals_fix = [s.get(dist, np.nan) for s in spaceRT_fixed_all]
    vals_fix = [v for v in vals_fix if not np.isnan(v)]

    ax.boxplot(
        vals_fix,
        positions=[x[i]],
        widths=bar_width * 0.9,
        patch_artist=True,
        boxprops=dict(facecolor='none', color='black', linewidth=0.3),
        medianprops=dict(color='black', linewidth=0.3),
        whiskerprops=dict(color='black', linewidth=0.3),
        capprops=dict(color='black', linewidth=0.3),
        showfliers=False
    )

    ax.scatter(np.random.normal(x[i], .02, len(vals_fix)), vals_fix,
               color=colors[i], s=4, alpha=.6, edgecolors='none')

# --- cosmetics ---

ax.set_ylabel('flip RT (ms)', fontsize=12, fontweight='bold')

ax.set_xticks(x)
ax.set_xticklabels(distributions_to_show, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.suptitle("mean flip RT", fontsize=14, fontweight='bold')
plt.tight_layout()


plt.savefig(os.path.join(output_folder, "spaceRT_mean.pdf"),
            format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "spaceRT_mean.svg"),
            format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_25696\2064503589.py:57: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


<h2 style="color: green;"> arrow RT</h2>

In [47]:
distributions_to_show = ['uniform', 'low', 'high']
colors = ['#808080', '#ff7f0e', '#2ca02c']
bar_width = 0.35

x = np.arange(len(distributions_to_show))
fig, ax = plt.subplots(figsize=(4, 3), dpi=130)

# Combine both groups
all_participants = dataframes + dataframes_epileptic

for i, dist in enumerate(distributions_to_show):
    # Fixed block
    fixed_vals = [
        pd.to_numeric(
            df.loc[(df['block_type'] == 'fix') & (df['distribution'] == dist), 'arrowRT'],
            errors='coerce'
        ).mean()
        for df in all_participants
    ]
    ax.boxplot(
        fixed_vals,
        positions=[x[i] - bar_width / 2],
        widths=bar_width * 0.9,
        patch_artist=True,
        boxprops=dict(facecolor='none', color='black'),
        medianprops=dict(color='black'),
        whiskerprops=dict(color='black'),
        capprops=dict(color='black'),
        showfliers=False
    )
    jitter_x = np.random.normal(x[i] - bar_width / 2, 0.02, size=len(fixed_vals))
    ax.scatter(jitter_x, fixed_vals, s=5, alpha=0.6, color=colors[i], edgecolors='none')

    # Mixed block
    mixed_vals = [
        pd.to_numeric(
            df.loc[(df['block_type'] == 'mix') & (df['distribution'] == dist), 'arrowRT'],
            errors='coerce'
        ).mean()
        for df in all_participants
    ]
    ax.boxplot(
        mixed_vals,
        positions=[x[i] + bar_width / 2],
        widths=bar_width * 0.9,
        patch_artist=True,
        boxprops=dict(facecolor='none', color='black'),
        medianprops=dict(color='black'),
        whiskerprops=dict(color='black'),
        capprops=dict(color='black'),
        showfliers=False
    )
    jitter_x = np.random.normal(x[i] + bar_width / 2, 0.02, size=len(mixed_vals))
    ax.scatter(jitter_x, mixed_vals, s=5, alpha=0.6, color=colors[i], edgecolors='none')

# Add x-axis labels
for i, dist in enumerate(distributions_to_show):
    ax.text(
        x[i] - bar_width / 2,
        -0.02,
        'fix',
        ha='center',
        va='top',
        fontsize=10,
        transform=ax.get_xaxis_transform(),
    )
    ax.text(
        x[i] + bar_width / 2,
        -0.02,
        'mix',
        ha='center',
        va='top',
        fontsize=10,
        transform=ax.get_xaxis_transform(),
    )
    ax.text(
        x[i],
        -0.12,
        dist,
        ha='center',
        va='top',
        fontsize=12,
        fontweight='bold',
        transform=ax.get_xaxis_transform(),
    )

ax.set_ylabel('arrowRT')
ax.set_title(',ean arrowRT')
ax.set_xticks([])

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
# plt.savefig(os.path.join(output_folder, "arrowRT_combined.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
# plt.savefig(os.path.join(output_folder, "arrowRT_combined.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()

C:\Users\Nill\AppData\Local\Temp\1\ipykernel_25696\594849674.py:97: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


<h2 style="color: green;">(2_1) arrowRT for different card number </h2>

In [48]:

# -----------------------------------------------
# Define color scheme
distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# Example: ensure these exist in your environment
# dataframes = [...]              # list of healthy DFs
# dataframes_epileptic = [...]    # list of epileptic DFs
# output_folder = "path/to/out"   # output directory

# -----------------------------------------------
# Function to compute combined arrowRT
def compute_combined_arrowRT(dataframes):
    arrowRT_by_card_block = []
    for df in dataframes:
        df = df.copy()
        df['arrowRT'] = pd.to_numeric(df['arrowRT'], errors='coerce')
        df['myCard'] = pd.to_numeric(df['myCard'], errors='coerce').astype('Int64')
        df = df.dropna(subset=['arrowRT', 'myCard'])
        df['myCard'] = df['myCard'].astype(int)
        arrow_rt_means = (
            df.groupby(['myCard', 'distribution', 'block_type'], observed=True)['arrowRT']
              .mean()
              .reset_index()
        )
        arrowRT_by_card_block.append(arrow_rt_means)
    return pd.concat(arrowRT_by_card_block, ignore_index=True)

# -----------------------------------------------
# Process both healthy and epileptic data
combined_arrowRT_all = compute_combined_arrowRT(dataframes + dataframes_epileptic)

# Filter only 'fix' block_type
combined_arrowRT_all = combined_arrowRT_all[combined_arrowRT_all['block_type'] == 'fix']

# -----------------------------------------------
# Create 3 subplots: one for each distribution
fig, axes = plt.subplots(1, 3, figsize=(8, 3.5), dpi=1200, sharey=True)

for i, dist in enumerate(distributions):
    data = combined_arrowRT_all[combined_arrowRT_all['distribution'] == dist]
    ax = axes[i]

    # Box data only for positions present in data
    positions = sorted(data['myCard'].unique())
    box_data = [data.loc[data['myCard'] == pos, 'arrowRT'] for pos in positions]

    # --- Boxplot (matplotlib)
    if len(box_data) > 0:
        ax.boxplot(
            box_data,
            positions=positions,
            widths=0.6,
            patch_artist=True,
            boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.3),
            medianprops=dict(color='black', linewidth=0.3),
            whiskerprops=dict(color='black', linewidth=0.3),
            capprops=dict(color='black', linewidth=0.3),
            flierprops=dict(marker='', linewidth=0.3),
        )

    # --- Strip plot
    for pos in positions:
        y_vals = data.loc[data['myCard'] == pos, 'arrowRT'].values
        if y_vals.size == 0:
            continue
        x_jittered = np.random.normal(loc=pos, scale=0.08, size=y_vals.size)
        ax.scatter(x_jittered, y_vals, color=colors[dist], s=9, alpha=0.6, linewidth=0)

    # --- Force x-axis ticks and labels to 1..9
    ax.set_xticks(range(1, 10))
    ax.set_xticklabels(range(1, 10))

    ax.set_title(f'{dist}', fontsize=12, fontweight='bold')
    ax.set_xlabel('myCard', fontsize=12, fontweight='bold')
    if i == 0:
        ax.set_ylabel('arrow RT (ms)', fontsize=12, fontweight='bold')
    else:
        ax.set_ylabel('')

    # Clean look
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# -----------------------------------------------
# Final layout and save
plt.tight_layout(rect=[0, 0, 1, 0.93])

os.makedirs(output_folder, exist_ok=True)
plt.savefig(os.path.join(output_folder, "arrowRT_fix_combined.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "arrowRT_fix_combined.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_25696\1235998301.py:38: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots(1, 3, figsize=(8, 3.5), dpi=1200, sharey=True)
C:\Users\Nill\AppData\Local\Temp\1\ipykernel_25696\1235998301.py:93: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


## arrow RT fix mix separated

In [49]:
# -----------------------------------------------
# Define color scheme
distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# Example: ensure these exist in your environment
# dataframes = [...]              # list of healthy DFs
# dataframes_epileptic = [...]    # list of epileptic DFs
# output_folder = "path/to/out"   # output directory

# -----------------------------------------------
# Function to compute combined arrowRT
def compute_combined_arrowRT(dataframes):
    arrowRT_by_card_block = []
    for df in dataframes:
        df = df.copy()
        df['arrowRT'] = pd.to_numeric(df['arrowRT'], errors='coerce')
        df['myCard'] = pd.to_numeric(df['myCard'], errors='coerce').astype('Int64')
        df = df.dropna(subset=['arrowRT', 'myCard'])
        df['myCard'] = df['myCard'].astype(int)
        arrow_rt_means = (
            df.groupby(['myCard', 'distribution', 'block_type'], observed=True)['arrowRT']
              .mean()
              .reset_index()
        )
        arrowRT_by_card_block.append(arrow_rt_means)
    return pd.concat(arrowRT_by_card_block, ignore_index=True)

# -----------------------------------------------
# Process both healthy and epileptic data
combined_arrowRT_all = compute_combined_arrowRT(dataframes + dataframes_epileptic)

# -----------------------------------------------
# Create subplots: 2 rows (fix, mix) × 3 cols (distributions)
fig, axes = plt.subplots(2, 3, figsize=(9, 6), dpi=1200, sharey=True)

block_types = ['fix', 'mix']

for row_idx, block in enumerate(block_types):
    sub_df = combined_arrowRT_all[combined_arrowRT_all['block_type'] == block]

    for col_idx, dist in enumerate(distributions):
        data = sub_df[sub_df['distribution'] == dist]
        ax = axes[row_idx, col_idx]

        positions = sorted(data['myCard'].unique())
        box_data = [data.loc[data['myCard'] == pos, 'arrowRT'] for pos in positions]

        # --- Boxplot
        if len(box_data) > 0:
            ax.boxplot(
                box_data,
                positions=positions,
                widths=0.6,
                patch_artist=True,
                boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.3),
                medianprops=dict(color='black', linewidth=0.3),
                whiskerprops=dict(color='black', linewidth=0.3),
                capprops=dict(color='black', linewidth=0.3),
                flierprops=dict(marker='', linewidth=0.3),
            )

        # --- Scatter (jittered)
        for pos in positions:
            y_vals = data.loc[data['myCard'] == pos, 'arrowRT'].values
            if y_vals.size == 0:
                continue
            x_jittered = np.random.normal(loc=pos, scale=0.08, size=y_vals.size)
            ax.scatter(x_jittered, y_vals, color=colors[dist], s=9, alpha=0.6, linewidth=0)

        # --- Force x-axis ticks 1..9
        ax.set_xticks(range(1, 10))
        ax.set_xticklabels(range(1, 10))

        # Titles and labels
        if row_idx == 0:
            ax.set_title(f'{dist}', fontsize=12, fontweight='bold')
        if col_idx == 0:
            ax.set_ylabel(f'{block}\narrow RT (ms)', fontsize=12, fontweight='bold')
        else:
            ax.set_ylabel('')
        ax.set_xlabel('myCard', fontsize=12, fontweight='bold')

        # Clean look
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# -----------------------------------------------
# Final layout and save
plt.tight_layout(rect=[0, 0, 1, 0.95])

os.makedirs(output_folder, exist_ok=True)
plt.savefig(os.path.join(output_folder, "arrowRT_fix_mix.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "arrowRT_fix_mix.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_25696\1707275598.py:96: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


## arrowRT curves

In [50]:
# -----------------------------------------------
# Define color scheme
distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# Example: ensure these exist in your environment
# dataframes = [...]              # list of healthy DFs
# dataframes_epileptic = [...]    # list of epileptic DFs
# output_folder = "path/to/out"   # output directory

# -----------------------------------------------
# Function to compute combined arrowRT (mean per participant × myCard × dist × block)
def compute_combined_arrowRT(dataframes):
    arrowRT_by_card_block = []
    for pid, df in enumerate(dataframes):
        df = df.copy()
        df['arrowRT'] = pd.to_numeric(df['arrowRT'], errors='coerce')
        df['myCard'] = pd.to_numeric(df['myCard'], errors='coerce').astype('Int64')
        df = df.dropna(subset=['arrowRT', 'myCard'])
        df['myCard'] = df['myCard'].astype(int)
        arrow_rt_means = (
            df.groupby(['myCard', 'distribution', 'block_type'], observed=True)['arrowRT']
              .mean()
              .reset_index()
        )
        arrow_rt_means['participant'] = pid  # tag participant
        arrowRT_by_card_block.append(arrow_rt_means)
    return pd.concat(arrowRT_by_card_block, ignore_index=True)

# -----------------------------------------------
# Process both healthy and epileptic data
combined_arrowRT_all = compute_combined_arrowRT(dataframes + dataframes_epileptic)

# -----------------------------------------------
# Create subplots: 2 rows (fix, mix) × 3 cols (distributions)
fig, axes = plt.subplots(2, 3, figsize=(9, 6), dpi=1200, sharey=True)

block_types = ['fix', 'mix']

for row_idx, block in enumerate(block_types):
    sub_df = combined_arrowRT_all[combined_arrowRT_all['block_type'] == block]

    for col_idx, dist in enumerate(distributions):
        data = sub_df[sub_df['distribution'] == dist]
        ax = axes[row_idx, col_idx]

        # --- Compute grand mean + SEM ---
        mean_curve = (
            data.groupby('myCard', observed=True)['arrowRT']
                .agg(['mean', 'sem'])
                .reset_index()
        )

        if not mean_curve.empty:
            x = mean_curve['myCard'].values
            y = mean_curve['mean'].values

            # --- Polynomial fit (degree 3) ---
            coeffs = np.polyfit(x, y, deg=3)
            poly = np.poly1d(coeffs)
            x_fit = np.linspace(min(x), max(x), 200)
            y_fit = poly(x_fit)

            # Plot fitted curve (dashed)
            ax.plot(
                x_fit, y_fit,
                color=colors[dist],
                linestyle='--',
                linewidth=1,
                alpha=0.8
            )

            # Plot mean markers with SEM error bars
            ax.errorbar(
                x,
                y,
                yerr=mean_curve['sem'].values,
                color=colors[dist],
                fmt='o',
                markersize=4,
                capsize=3,
                elinewidth=1,
                linewidth=0
            )

        # --- Force x-axis ticks 1..9
        ax.set_xticks(range(1, 10))
        ax.set_xticklabels(range(1, 10))

        # Titles and labels
        if row_idx == 0:
            ax.set_title(f'{dist}', fontsize=12, fontweight='bold')
        if col_idx == 0:
            ax.set_ylabel(f'{block}\narrow RT (ms)', fontsize=12, fontweight='bold')
        else:
            ax.set_ylabel('')
        ax.set_xlabel('myCard', fontsize=12, fontweight='bold')

        # Clean look
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# -----------------------------------------------
# Final layout and save
plt.tight_layout(rect=[0, 0, 1, 0.95])

os.makedirs(output_folder, exist_ok=True)
plt.savefig(os.path.join(output_folder, "arrowRT_fix_mix_curve.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "arrowRT_fix_mix_curve.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_25696\3524827251.py:111: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


In [51]:

distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# -------------------------------------------------
# Gaussian function
def gaussian(x, A, mu, sigma, baseline):
    return A * np.exp(-(x - mu)**2 / (2 * sigma**2)) + baseline

# -------------------------------------------------
# Function to compute combined arrowRT (mean per participant × myCard × dist × block)
def compute_combined_arrowRT(dataframes):
    arrowRT_by_card_block = []
    for pid, df in enumerate(dataframes):
        df = df.copy()
        df['arrowRT'] = pd.to_numeric(df['arrowRT'], errors='coerce')
        df['myCard'] = pd.to_numeric(df['myCard'], errors='coerce').astype('Int64')
        df = df.dropna(subset=['arrowRT', 'myCard'])
        df['myCard'] = df['myCard'].astype(int)
        arrow_rt_means = (
            df.groupby(['myCard', 'distribution', 'block_type'], observed=True)['arrowRT']
              .mean()
              .reset_index()
        )
        arrow_rt_means['participant'] = pid  # tag participant
        arrowRT_by_card_block.append(arrow_rt_means)
    return pd.concat(arrowRT_by_card_block, ignore_index=True)

# -------------------------------------------------
# Process both healthy and epileptic data
combined_arrowRT_all = compute_combined_arrowRT(dataframes + dataframes_epileptic)

# -------------------------------------------------
# Create subplots: 2 rows (fix, mix) × 3 cols (distributions)
fig, axes = plt.subplots(2, 3, figsize=(9, 6), dpi=1200, sharey=True)

block_types = ['fix', 'mix']

for row_idx, block in enumerate(block_types):
    sub_df = combined_arrowRT_all[combined_arrowRT_all['block_type'] == block]

    for col_idx, dist in enumerate(distributions):
        data = sub_df[sub_df['distribution'] == dist]
        ax = axes[row_idx, col_idx]

        # --- Compute grand mean + SEM ---
        mean_curve = (
            data.groupby('myCard', observed=True)['arrowRT']
                .agg(['mean', 'sem'])
                .reset_index()
        )

        if not mean_curve.empty:
            x = mean_curve['myCard'].values
            y = mean_curve['mean'].values

            try:
                # Initial guesses: amplitude=max-min, mu=midpoint, sigma=2, baseline=min
                p0 = [y.max() - y.min(), np.mean(x), 2, y.min()]
                popt, _ = curve_fit(gaussian, x, y, p0=p0, maxfev=5000)

                x_fit = np.linspace(min(x), max(x), 200)
                y_fit = gaussian(x_fit, *popt)

                # Plot fitted Gaussian (dashed)
                ax.plot(
                    x_fit, y_fit,
                    color=colors[dist],
                    linestyle='--',
                    linewidth=1,
                    alpha=0.8
                )
            except RuntimeError:
                print(f"Gaussian fit failed for block={block}, dist={dist}")

            # Plot mean markers with SEM error bars
            ax.errorbar(
                x,
                y,
                yerr=mean_curve['sem'].values,
                color=colors[dist],
                fmt='o',
                markersize=4,
                capsize=3,
                elinewidth=1,
                linewidth=0
            )

        # --- Force x-axis ticks 1..9
        ax.set_xticks(range(1, 10))
        ax.set_xticklabels(range(1, 10))

        # Titles and labels
        if row_idx == 0:
            ax.set_title(f'{dist}', fontsize=12, fontweight='bold')
        if col_idx == 0:
            ax.set_ylabel(f'{block}\narrow RT (ms)', fontsize=12, fontweight='bold')
        else:
            ax.set_ylabel('')
        ax.set_xlabel('myCard', fontsize=12, fontweight='bold')

        # Clean look
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# -------------------------------------------------
# Final layout and save
plt.tight_layout(rect=[0, 0, 1, 0.95])

os.makedirs(output_folder, exist_ok=True)
plt.savefig(os.path.join(output_folder, "arrowRT_fix_mix_curve_gaussian.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "arrowRT_fix_mix_curve_gaussian.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_25696\359409580.py:113: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


In [52]:
distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# -------------------------------------------------
# Gaussian function
def gaussian(x, A, mu, sigma, baseline):
    return A * np.exp(-(x - mu)**2 / (2 * sigma**2)) + baseline

# -------------------------------------------------

def compute_combined_arrowRT(dataframes):
    arrowRT_by_card_block = []
    for pid, df in enumerate(dataframes):
        df = df.copy()
        df['arrowRT'] = pd.to_numeric(df['arrowRT'], errors='coerce')
        df['myCard'] = pd.to_numeric(df['myCard'], errors='coerce').astype('Int64')
        df = df.dropna(subset=['arrowRT', 'myCard'])
        df['myCard'] = df['myCard'].astype(int)
        arrow_rt_means = (
            df.groupby(['myCard', 'distribution', 'block_type'], observed=True)['arrowRT']
              .mean()
              .reset_index()
        )
        arrow_rt_means['participant'] = pid  # tag participant
        arrowRT_by_card_block.append(arrow_rt_means)
    return pd.concat(arrowRT_by_card_block, ignore_index=True)

# -------------------------------------------------
# Process both healthy and epileptic data
combined_arrowRT_all = compute_combined_arrowRT(dataframes + dataframes_epileptic)

# -------------------------------------------------
# Create subplots: 1 row × 2 cols (fix, mix)
fig, axes = plt.subplots(1, 2, figsize=(5, 3), dpi=1200, sharey=True)

block_types = ['fix', 'mix']

for ax, block in zip(axes, block_types):
    sub_df = combined_arrowRT_all[combined_arrowRT_all['block_type'] == block]

    for dist in distributions:
        data = sub_df[sub_df['distribution'] == dist]

        # --- Compute grand mean + SEM ---
        mean_curve = (
            data.groupby('myCard', observed=True)['arrowRT']
                .agg(['mean', 'sem'])
                .reset_index()
        )

        if not mean_curve.empty:
            x = mean_curve['myCard'].values
            y = mean_curve['mean'].values

            try:
                # Initial guesses: amplitude=max-min, mu=midpoint, sigma=2, baseline=min
                p0 = [y.max() - y.min(), np.mean(x), 2, y.min()]
                popt, _ = curve_fit(gaussian, x, y, p0=p0, maxfev=5000)

                x_fit = np.linspace(min(x), max(x), 200)
                y_fit = gaussian(x_fit, *popt)

                # Plot fitted Gaussian (dashed)
                ax.plot(
                    x_fit, y_fit,
                    color=colors[dist],
                    linestyle='--',
                    linewidth=0.5,
                    alpha=0.8,
                    label=f"{dist} fit" if block == 'fix' else None  # legend only once
                )
            except RuntimeError:
                print(f"Gaussian fit failed for block={block}, dist={dist}")

            # Plot mean markers with SEM error bars
            ax.errorbar(
                x,
                y,
                yerr=mean_curve['sem'].values,
                color=colors[dist],
                fmt='o',
                markersize=2,
                capsize=2.5,
                elinewidth=0.5,
                capthick=0.2,   # <- thinner cap lines
                linewidth=0.5,
                label=f"{dist} mean" if block == 'fix' else None
            )

    # --- Force x-axis ticks 1..9
    ax.set_xticks(range(1, 10))
    ax.set_xticklabels(range(1, 10))

    # Titles and labels
    ax.set_title(f'{block}', fontsize=12)
    ax.set_xlabel('card number', fontsize=12)
    if block == 'fix':
        ax.set_ylabel('choice RT (ms)', fontsize=12, fontweight='bold')

    # Clean look
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))



# -------------------------------------------------
# Final layout and save
plt.tight_layout(rect=[0, 0, 1, 0.95])

os.makedirs(output_folder, exist_ok=True)
plt.savefig(os.path.join(output_folder, "arrowRT_fix_mix_curve_gaussian_smaller_plot.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "arrowRT_fix_mix_curve_gaussian_smaller_plot.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_25696\2428994883.py:114: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


# gaussian params

In [58]:
import numpy as np
import pandas as pd
import warnings
from scipy.optimize import curve_fit, OptimizeWarning
import os

# -------------------------------------------------
# Gaussian function
def gaussian(x, A, mu, sigma, baseline):
    return A * np.exp(-(x - mu)**2 / (2 * sigma**2)) + baseline

# -------------------------------------------------
# Numerical skewness calculation for fitted Gaussian
def compute_skewness(A, mu, sigma, baseline, x_min=1, x_max=9, n_points=1000):
    """
    Compute skewness of the fitted Gaussian curve numerically.
    Integration is approximated over the support [x_min, x_max].
    """
    x = np.linspace(x_min, x_max, n_points)
    y = gaussian(x, A, mu, sigma, baseline)
    
    # Normalize to get a probability-like distribution
    total = np.trapz(y, x)
    if total <= 0:
        return np.nan
    probs = y / total
    
    # Mean
    mean = np.trapz(x * probs, x)
    # Variance
    var = np.trapz((x - mean)**2 * probs, x)
    sigma_eff = np.sqrt(var)
    if sigma_eff == 0:
        return np.nan
    # Skewness
    skew = np.trapz((x - mean)**3 * probs, x) / (sigma_eff**3)
    return skew

# -------------------------------------------------
# Config
distributions = ['uniform', 'low', 'high']
blocks = ['fix', 'mix']

# -------------------------------------------------
def fit_subject_gaussian(df, dist, block):
    sub_df = df[(df['distribution'] == dist) & (df['block_type'] == block)].copy()
    if sub_df.empty:
        return None

    # ensure numeric
    sub_df['arrowRT'] = pd.to_numeric(sub_df['arrowRT'], errors='coerce')
    sub_df = sub_df.dropna(subset=['arrowRT'])

    # aggregate to card means
    g = sub_df.groupby('myCard', observed=True)['arrowRT'].mean().reset_index()
    x = g['myCard'].values
    y = g['arrowRT'].values

    if len(x) < 3:
        return None

    # Initial guess
    p0 = [max(y) - min(y), np.mean(x), 2, min(y)]
    bounds = ([0, 1, 0.1, 0], [np.inf, 9, 10, np.inf])

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", OptimizeWarning)
        try:
            popt, _ = curve_fit(gaussian, x, y, p0=p0, bounds=bounds, maxfev=5000)
            A, mu, sigma, baseline = popt

            # Post-hoc filter
            if baseline < 0:
                return None

            # Compute skewness
            skew = compute_skewness(A, mu, sigma, baseline)

            return popt, skew
        except (RuntimeError, ValueError):
            return None

# -------------------------------------------------
# Main loop
fit_rows = []
all_dfs = dataframes + dataframes_epileptic  # 47 subjects

for pid, df in enumerate(all_dfs):
    for block in blocks:
        for dist in distributions:
            result = fit_subject_gaussian(df, dist, block)
            if result is not None:
                popt, skew = result
                fit_rows.append({
                    'subject_idx': pid,
                    'block': block,
                    'distribution': dist,
                    'A': popt[0],
                    'mu': popt[1],
                    'sigma': popt[2],
                    'baseline': popt[3],
                    'skewness': skew,
                })
            else:
                print(f"[Rejected] subject {pid}, block={block}, dist={dist}")

# -------------------------------------------------
# Collect
gaussian_fit_df = pd.DataFrame(fit_rows)


[Rejected] subject 1, block=fix, dist=high
[Rejected] subject 8, block=fix, dist=uniform
[Rejected] subject 8, block=mix, dist=uniform
[Rejected] subject 9, block=fix, dist=uniform
[Rejected] subject 10, block=fix, dist=low
[Rejected] subject 10, block=fix, dist=high
[Rejected] subject 14, block=fix, dist=uniform
[Rejected] subject 15, block=mix, dist=low
[Rejected] subject 17, block=mix, dist=low
[Rejected] subject 24, block=mix, dist=low
[Rejected] subject 26, block=fix, dist=uniform
[Rejected] subject 28, block=fix, dist=low
[Rejected] subject 33, block=fix, dist=uniform
[Rejected] subject 34, block=fix, dist=uniform
[Rejected] subject 34, block=fix, dist=low
[Rejected] subject 34, block=mix, dist=uniform
[Rejected] subject 34, block=mix, dist=high
[Rejected] subject 38, block=fix, dist=uniform
[Rejected] subject 44, block=fix, dist=high


In [60]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator
from scipy.stats import mannwhitneyu
import os

# -------------------------------------------------
# Colors & order
distributions = ['uniform', 'low', 'high']
blocks = ['fix', 'mix']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# -------------------------------------------------
def gaussian_skewness(mu, sigma):
    return (mu-5) / sigma

# -------------------------------------------------
# Prepare data: long-format
df_plot = gaussian_fit_df.copy()
# df_plot['skewness'] = gaussian_skewness(df_plot['mu'], df_plot['sigma'])

df_plot['skewness'] = df_plot['skewness'].astype(float)
# -------------------------------------------------
# Significance bar helper (fixed top position)
def add_sig_bar(ax, pos1, pos2, y_top, pval, tick_height=0.2, text_offset=0.05, extra_offset=0.2):
    """
    Draws a significance bar higher by adding extra_offset
    """

    # shift everything upward
    y_top = y_top + extra_offset

    # vertical ticks
    ax.plot([pos1, pos1], [y_top - tick_height, y_top], lw=0.5, c='black')
    ax.plot([pos2, pos2], [y_top - tick_height, y_top], lw=0.5, c='black')
    # horizontal connector
    ax.plot([pos1, pos2], [y_top, y_top], lw=0.5, c='black')

    # significance stars
    if pval < 0.001:
        sig_text = '***'
    elif pval < 0.01:
        sig_text = '**'
    elif pval < 0.05:
        sig_text = '*'
    else:
        sig_text = 'n.s.'

    ax.text((pos1 + pos2) / 2, y_top + text_offset, sig_text,
            ha='center', va='bottom', fontsize=8)


# -------------------------------------------------
# Set up figure: 1 row × 3 cols
fig, axes = plt.subplots(1, 2, figsize=(4,2), dpi=1200, sharex=True)

metrics = ['mu',  'skewness']
ylabels = [r'$\mu$', 'skewness']
ylims = [(0, 10), (-1,1)]

rng = np.random.default_rng(42)
group_gap = 2.2
pair_offset = 0.35

for ax, metric, ylabel, ylim in zip(axes, metrics, ylabels, ylims):
    box_data, box_positions, box_colors = [], [], []
    x_cursor = 0.0

    ymin, ymax = ylim
    yspan = ymax - ymin
    # fixed bar top for all sig bars in this subplot
    bar_y_fixed = ymax - 0.05 * yspan   # 5% below the top

    for dist in distributions:
        pos_fix = x_cursor - pair_offset
        pos_mix = x_cursor + pair_offset

        for blk, pos in zip(blocks, [pos_fix, pos_mix]):
            sub = df_plot[(df_plot['distribution'] == dist) &
                          (df_plot['block'] == blk)][metric].astype(float).values
            if sub.size == 0:
                sub = np.array([np.nan])
            box_data.append(sub)
            box_positions.append(pos)
            box_colors.append(colors[dist])

        # significance test fix vs mix
        vals_fix = df_plot[(df_plot['distribution'] == dist) &
                           (df_plot['block'] == 'fix')][metric].dropna().values
        vals_mix = df_plot[(df_plot['distribution'] == dist) &
                           (df_plot['block'] == 'mix')][metric].dropna().values

        if len(vals_fix) > 0 and len(vals_mix) > 0:
            stat, pval = mannwhitneyu(vals_fix, vals_mix, alternative='two-sided')
        else:
            pval = 1.0

        # all bars use the same bar_y_fixed
        add_sig_bar(ax, pos_fix, pos_mix, bar_y_fixed, pval)

        x_cursor += group_gap

    # Plot boxplot
    bp = ax.boxplot(
        box_data, positions=box_positions, widths=0.5, patch_artist=True, manage_ticks=False,
        boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.5),
        medianprops=dict(color='black', linewidth=0.5),
        whiskerprops=dict(color='black', linewidth=0.5),
        capprops=dict(color='black', linewidth=0.5),
        flierprops=dict(marker='', linewidth=0.5),
    )
    for patch in bp['boxes']:
        patch.set_edgecolor('black')

    # Add jittered scatter for participants
    for pos, vals, c in zip(box_positions, box_data, box_colors):
        vals = vals[~np.isnan(vals)]
        if vals.size == 0:
            continue
        xs = rng.normal(loc=pos, scale=0.035, size=vals.size)
        ax.scatter(xs, vals, s=5, alpha=0.5, linewidth=0, color=c, edgecolors='none')

    # Labels and styling
    ax.set_ylabel(ylabel, fontsize=8)
    ax.set_ylim(ylim)  # apply fixed limits
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    tick_labels_lvl1 = []
    for _dist in distributions:
        tick_labels_lvl1.extend(['fix', 'mix'])
    ax.set_xticks(box_positions)
    ax.set_xticklabels(tick_labels_lvl1, fontsize=8)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

plt.subplots_adjust(hspace=0.45, bottom=0.2, wspace=0.35)

# Save
os.makedirs(output_folder, exist_ok=True)
plt.savefig(os.path.join(output_folder, "arrowRT_fix_mix_curve_gaussian_params.pdf"),
            format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "arrowRT_fix_mix_curve_gaussian_params.svg"),
            format="svg", dpi=1200, bbox_inches="tight")
plt.show()



C:\Users\Nill\AppData\Local\Temp\1\ipykernel_25696\3827427232.py:143: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


In [61]:

import pandas as pd

# Store results
results = []

for ax, metric, ylabel, ylim in zip(axes, metrics, ylabels, ylims):
    box_data, box_positions, box_colors = [], [], []
    x_cursor = 0.0

    ymin, ymax = ylim
    yspan = ymax - ymin
    bar_y_fixed = ymax - 0.05 * yspan   # 5% below the top

    for dist in distributions:
        pos_fix = x_cursor - pair_offset
        pos_mix = x_cursor + pair_offset

        for blk, pos in zip(blocks, [pos_fix, pos_mix]):
            sub = df_plot[(df_plot['distribution'] == dist) &
                          (df_plot['block'] == blk)][metric].astype(float).values
            if sub.size == 0:
                sub = np.array([np.nan])
            box_data.append(sub)
            box_positions.append(pos)
            box_colors.append(colors[dist])

        # significance test fix vs mix
        vals_fix = df_plot[(df_plot['distribution'] == dist) &
                           (df_plot['block'] == 'fix')][metric].dropna().values
        vals_mix = df_plot[(df_plot['distribution'] == dist) &
                           (df_plot['block'] == 'mix')][metric].dropna().values

        if len(vals_fix) > 0 and len(vals_mix) > 0:
            stat, pval = mannwhitneyu(vals_fix, vals_mix, alternative='two-sided')
        else:
            pval = 1.0

        # record into table
        results.append({
            "metric": metric,
            "distribution": dist,
            "p_value": pval
        })

        add_sig_bar(ax, pos_fix, pos_mix, bar_y_fixed, pval)
        x_cursor += group_gap

# Convert to DataFrame
pval_table = pd.DataFrame(results)
print(pval_table)



     metric distribution   p_value
0        mu      uniform  0.734630
1        mu          low  0.064532
2        mu         high  0.001095
3  skewness      uniform  0.643912
4  skewness          low  0.038097
5  skewness         high  0.000870


## explanation of the code above:
I have fitted a gaussian to every single participant and then took the mean of all of those and then got the mean of all the gaussian parameters and then reconstruct it

In [56]:
# # -----------------------------------------------
# # Define color scheme
# distributions = ['uniform', 'low', 'high']
# colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}


# # -----------------------------------------------
# # Function to compute combined arrowRT (mean per participant × myCard × dist × block)
# def compute_combined_arrowRT(dataframes):
#     arrowRT_by_card_block = []
#     for pid, df in enumerate(dataframes):
#         df = df.copy()
#         df['arrowRT'] = pd.to_numeric(df['arrowRT'], errors='coerce')
#         df['myCard'] = pd.to_numeric(df['myCard'], errors='coerce').astype('Int64')
#         df = df.dropna(subset=['arrowRT', 'myCard'])
#         df['myCard'] = df['myCard'].astype(int)
#         arrow_rt_means = (
#             df.groupby(['myCard', 'distribution', 'block_type'], observed=True)['arrowRT']
#               .mean()
#               .reset_index()
#         )
#         arrow_rt_means['participant'] = pid  # tag participant
#         arrowRT_by_card_block.append(arrow_rt_means)
#     return pd.concat(arrowRT_by_card_block, ignore_index=True)

# # -----------------------------------------------
# # Process both healthy and epileptic data
# combined_arrowRT_all = compute_combined_arrowRT(dataframes + dataframes_epileptic)

# # -----------------------------------------------
# # Create subplots: 2 rows (fix, mix) × 3 cols (distributions)
# fig, axes = plt.subplots(2, 3, figsize=(9, 6), dpi=1200, sharey=True)

# block_types = ['fix', 'mix']

# for row_idx, block in enumerate(block_types):
#     sub_df = combined_arrowRT_all[combined_arrowRT_all['block_type'] == block]

#     for col_idx, dist in enumerate(distributions):
#         data = sub_df[sub_df['distribution'] == dist]
#         ax = axes[row_idx, col_idx]

#         # --- Compute grand mean + SEM ---
#         mean_curve = (
#             data.groupby('myCard', observed=True)['arrowRT']
#                 .agg(['mean', 'sem'])   # <-- SEM instead of STD
#                 .reset_index()
#         )

#         if not mean_curve.empty:
#             ax.errorbar(
#                 mean_curve['myCard'],
#                 mean_curve['mean'],
#                 yerr=mean_curve['sem'],
#                 color=colors[dist],
#                 fmt='o',
#                 linestyle='--',
#                 markersize=4,
#                 linewidth=1,
#                 capsize=3,
#                 elinewidth=1
#             )

#         # --- Force x-axis ticks 1..9
#         ax.set_xticks(range(1, 10))
#         ax.set_xticklabels(range(1, 10))

#         # Titles and labels
#         if row_idx == 0:
#             ax.set_title(f'{dist}', fontsize=12, fontweight='bold')
#         if col_idx == 0:
#             ax.set_ylabel(f'{block}\narrow RT (ms)', fontsize=12, fontweight='bold')
#         else:
#             ax.set_ylabel('')
#         ax.set_xlabel('myCard', fontsize=12, fontweight='bold')

#         # Clean look
#         ax.spines['top'].set_visible(False)
#         ax.spines['right'].set_visible(False)
#         ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# # -----------------------------------------------
# # Final layout and save
# plt.tight_layout(rect=[0, 0, 1, 0.95])



# plt.savefig(os.path.join(output_folder, "arrowRT_fix_mix_curve_no_fit.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
# plt.savefig(os.path.join(output_folder, "arrowRT_fix_mix_curve_no_fit.svg"), format="svg", dpi=1200, bbox_inches="tight")
# plt.show()


## space RT fix and mix by distribution

In [57]:

# Define color scheme
distributions = ['uniform', 'low', 'high']
blocks = ['fix', 'mix']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# -------------------------------------------------
# Compute participant-level means (one mean per participant × dist × block)
def compute_participant_means_spaceRT(df_list, max_rt=1500, min_rt=1e-9):
    """
    Treat each DF in df_list as one participant.
    Returns a tidy DataFrame with columns:
    ['participant', 'distribution', 'block_type', 'spaceRT_mean'].
    Filters spaceRT to (min_rt, max_rt] and ignores myCard.
    """
    rows = []
    for pid, df in enumerate(df_list):
        d = df.copy()
        # clean
        d['spaceRT'] = pd.to_numeric(d['spaceRT'], errors='coerce')
        d = d.dropna(subset=['spaceRT', 'distribution', 'block_type'])
        d = d[(d['spaceRT'] > min_rt) & (d['spaceRT'] <= max_rt)]

        if d.empty:
            continue

        g = (
            d.groupby(['distribution', 'block_type'], observed=True)['spaceRT']
             .mean()
             .reset_index(name='spaceRT_mean')
        )
        g['participant'] = pid
        rows.append(g[['participant', 'distribution', 'block_type', 'spaceRT_mean']])

    if not rows:
        return pd.DataFrame(columns=['participant','distribution','block_type','spaceRT_mean'])

    out = pd.concat(rows, ignore_index=True)
    out['distribution'] = out['distribution'].astype('category')
    out['block_type']   = out['block_type'].astype('category')
    return out

# -------------------------------------------------
# Combine healthy + epileptic, then compute participant means
all_participant_means = pd.concat([
    compute_participant_means_spaceRT(dataframes),
    compute_participant_means_spaceRT(dataframes_epileptic)
], ignore_index=True)

# -------------------------------------------------
# Prepare plotting data: for each distribution, we create 2 boxes (fix, mix)
box_data = []         # list of 1D arrays (participant means) per cell
box_positions = []    # x positions
box_colors = []       # color by distribution
group_centers = []    # center of each dist group for 2nd-level label

group_gap = 2.2
pair_offset = 0.35
x_cursor = 0.0

for dist in distributions:
    pos_fix = x_cursor - pair_offset
    pos_mix = x_cursor + pair_offset
    group_centers.append(x_cursor)

    for blk, pos in zip(blocks, [pos_fix, pos_mix]):
        sub = all_participant_means[
            (all_participant_means['distribution'] == dist) &
            (all_participant_means['block_type'] == blk)
        ]['spaceRT_mean'].astype(float).values

        # keep alignment even if empty
        if sub.size == 0:
            sub = np.array([np.nan])

        box_data.append(sub)
        box_positions.append(pos)
        box_colors.append(colors[dist])

    x_cursor += group_gap

# -------------------------------------------------
# Plot
fig, ax = plt.subplots(figsize=(3,), dpi=1200)

bp = ax.boxplot(
    box_data,
    positions=box_positions,
    widths=0.5,
    patch_artist=True,
    manage_ticks=False,
    boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.5),
    medianprops=dict(color='black', linewidth=0.5),
    whiskerprops=dict(color='black', linewidth=0.5),
    capprops=dict(color='black', linewidth=0.5),
    flierprops=dict(marker='', linewidth=0.5),
)

# Keep outlines black; medians already set above
for patch in bp['boxes']:
    patch.set_edgecolor('black')

# Scatter: ONE point per participant per cell (those participant means)
rng = np.random.default_rng(42)
for pos, vals, c in zip(box_positions, box_data, box_colors):
    vals = vals[~np.isnan(vals)]
    if vals.size == 0:
        continue
    xs = rng.normal(loc=pos, scale=0.035, size=vals.size)
    ax.scatter(xs, vals, s=24, alpha=0.5, linewidth=0, color=c)

# Labels
ax.set_ylabel('flip RT (ms)', fontsize=12, fontweight='bold')

# Level 1 x-axis: fix/mix
tick_labels_lvl1 = []
for _dist in distributions:
    tick_labels_lvl1.extend(['fix', 'mix'])
ax.set_xticks(box_positions)
ax.set_xticklabels(tick_labels_lvl1, fontsize=11)

# Level 2 x-axis: distribution centered under each pair
ymin, ymax = ax.get_ylim()
yspan = ymax - ymin
label_y = ymin - 0.06 * yspan
for center, dist in zip(group_centers, distributions):
    ax.text(center, label_y, dist, ha='center', va='top',
            fontsize=11, fontweight='bold')

# Bottom margin for second label row
plt.subplots_adjust(bottom=0.22)

# Style
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.set_major_locator(MaxNLocator(nbins=6, integer=False))
ax.title.set_text('mean flip RT by distribution and block type')

# -------------------------------------------------
# Paired permutation test
# -------------------------------------------------
def paired_permutation_p_value(a, b, n_perm=10000, rng=None):
    """
    Two-sided paired permutation (sign-flip) test on differences (a - b).
    Returns p-value.
    """
    a = np.asarray(a, float)
    b = np.asarray(b, float)
    if a.size == 0 or b.size == 0:
        return np.nan
    d = a - b
    d = d[~np.isnan(d)]
    if d.size < 2:
        return np.nan
    obs = np.abs(np.mean(d))
    if rng is None:
        rng = np.random.default_rng()
    signs = rng.choice([-1.0, 1.0], size=(n_perm, d.size))
    perm_stats = np.abs((signs * d).mean(axis=1))
    p_val = (np.sum(perm_stats >= obs) + 1) / (n_perm + 1)
    return p_val

# -------------------------------------------------
# Stats: fix vs mix per distribution
paired_data = {}
raw_pvals = []
for dist in distributions:
    sub = all_participant_means[all_participant_means['distribution'] == dist]
    piv = sub.pivot_table(index='participant', columns='block_type', values='spaceRT_mean', aggfunc='mean')
    if 'fix' in piv.columns and 'mix' in piv.columns:
        valid = piv[['fix', 'mix']].dropna()
        a = valid['fix'].values
        b = valid['mix'].values
    else:
        a, b = np.array([]), np.array([])

    paired_data[dist] = (a, b)
    p = paired_permutation_p_value(a, b, n_perm=10000, rng=rng)
    raw_pvals.append(p)

# Bonferroni correction across the 3 tests
alpha = 0.05
m = 3
adj_pvals = [min(p*m, 1.0) if not np.isnan(p) else np.nan for p in raw_pvals]

# Annotate fix vs mix
ymin, ymax = ax.get_ylim()
yspan = ymax - ymin

for center, dist, p_adj in zip(group_centers, distributions, adj_pvals):
    local_vals = all_participant_means.loc[
        all_participant_means['distribution'] == dist, 'spaceRT_mean'
    ].astype(float).values
    local_max = np.nanmax(local_vals) if local_vals.size > 0 else ymax
    y_text = local_max + 0.06 * yspan
    label = "n.s."
    if not np.isnan(p_adj) and p_adj < alpha:
        label = "*"

    if y_text > ymax:
        ymax = y_text + 0.03 * yspan
        ax.set_ylim(ymin, ymax)
        yspan = ymax - ymin

    ax.text(center, y_text, label, ha='center', va='bottom',
            fontsize=11)

    pos_fix = box_positions[2*distributions.index(dist)]
    pos_mix = box_positions[2*distributions.index(dist)+1]
    y_bar = y_text - 0.015 * yspan
    ax.plot([pos_fix, pos_fix, pos_mix, pos_mix],
            [y_bar, y_bar+0.02*yspan, y_bar+0.02*yspan, y_bar],
            color='black', linewidth=0.5)

# -------------------------------------------------
# Stats among MIX distributions (uniform vs low vs high)
# -------------------------------------------------
mix_only = all_participant_means[all_participant_means['block_type'] == 'mix']
mix_piv = mix_only.pivot_table(index='participant', columns='distribution',
                               values='spaceRT_mean', aggfunc='mean')

pairs = [('uniform', 'low'), ('uniform', 'high'), ('low', 'high')]
raw_pvals_mix = []
for a_lab, b_lab in pairs:
    if a_lab in mix_piv.columns and b_lab in mix_piv.columns:
        valid = mix_piv[[a_lab, b_lab]].dropna()
        a = valid[a_lab].values
        b = valid[b_lab].values
    else:
        a, b = np.array([]), np.array([])
    p = paired_permutation_p_value(a, b, n_perm=10000, rng=rng)
    raw_pvals_mix.append(p)

adj_pvals_mix = [min(p*3, 1.0) if not np.isnan(p) else np.nan for p in raw_pvals_mix]


# Annotate MIX comparisons (uniform vs low vs high)
# -------------------------------------------------
ymin, ymax = ax.get_ylim()
yspan = ymax - ymin
mix_positions = {dist: box_positions[2*idx+1] for idx, dist in enumerate(distributions)}

# Offset step so each comparison gets its own vertical layer
bar_step = 0.12 * yspan   # vertical spacing between bars
text_offset = 0.02 * yspan

for i, (pair, p_adj) in enumerate(zip(pairs, adj_pvals_mix)):
    dist_a, dist_b = pair
    pos_a, pos_b = mix_positions[dist_a], mix_positions[dist_b]

    local_vals = mix_only[mix_only['distribution'].isin([dist_a, dist_b])]['spaceRT_mean'].astype(float).values
    local_max = np.nanmax(local_vals) if local_vals.size > 0 else ymax

    # Each new bar is stacked higher using i * bar_step
    y_bar = local_max + (i+1) * bar_step
    y_text = y_bar + text_offset

    ax.plot([pos_a, pos_a, pos_b, pos_b],
            [y_bar, y_bar+0.02*yspan, y_bar+0.02*yspan, y_bar],
            color='black', linewidth=0.5)

    label = "n.s."
    if not np.isnan(p_adj) and p_adj < alpha:
        label = "*"
    ax.text((pos_a+pos_b)/2, y_text, label, ha='center', va='bottom',
            fontsize=11)

    # Expand y-limits dynamically if needed
    if y_text > ymax:
        ymax = y_text + 0.05 * yspan
        ax.set_ylim(ymin, ymax)

# -------------------------------------------------
# Save
plt.tight_layout()
os.makedirs(output_folder, exist_ok=True)
plt.savefig(os.path.join(output_folder, "spaceRT_mean_fix_mix.pdf"),
            format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "spaceRT_mean_fix_mix.svg"),
            format="svg", dpi=1200, bbox_inches="tight")
plt.show()


TypeError: Bbox.from_bounds() missing 1 required positional argument: 'height'

# space RT and arrow RT across trials

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import os

# -------------------------------------------------
# Compute trial-wise mean RT across participants
def compute_trialwise_means(df_list, rt_col, max_rt=1500, min_rt=1e-9):
    """
    Returns DataFrame with trial index, mean, and SEM across participants.
    Each DF in df_list = one participant. Trial index assigned by row order.
    """
    trial_dfs = []
    for pid, df in enumerate(df_list):
        d = df.copy().reset_index(drop=True)

        if rt_col not in d.columns:
            continue

        d[rt_col] = pd.to_numeric(d[rt_col], errors='coerce')
        d = d.dropna(subset=[rt_col])
        d = d[(d[rt_col] > min_rt) & (d[rt_col] <= max_rt)]

        if d.empty:
            continue

        # assign trial index
        d['trial'] = np.arange(1, len(d) + 1)

        # keep participant-level data
        g = d[['trial', rt_col]].copy()
        g['participant'] = pid
        trial_dfs.append(g)

    if not trial_dfs:
        return pd.DataFrame(columns=['trial', f'{rt_col}_mean', f'{rt_col}_sem'])

    all_data = pd.concat(trial_dfs, ignore_index=True)

    # compute across participants
    summary = all_data.groupby('trial')[rt_col].agg(['mean', 'sem']).reset_index()
    summary.rename(columns={'mean': f'{rt_col}_mean', 'sem': f'{rt_col}_sem'}, inplace=True)
    return summary

# -------------------------------------------------
# Helper: moving average smoother
def smooth_series(y, window=10):
    return y.rolling(window=window, center=True, min_periods=1).mean()

# -------------------------------------------------
# Combine healthy + epileptic
combined_dfs = dataframes + dataframes_epileptic

space_summary = compute_trialwise_means(combined_dfs, 'spaceRT')
arrow_summary = compute_trialwise_means(combined_dfs, 'arrowRT')

# Apply smoothing
space_summary['spaceRT_mean_smooth'] = smooth_series(space_summary['spaceRT_mean'], window=10)
arrow_summary['arrowRT_mean_smooth'] = smooth_series(arrow_summary['arrowRT_mean'], window=10)

# Smooth SEM as well
space_summary['spaceRT_sem_smooth'] = smooth_series(space_summary['spaceRT_sem'], window=10)
arrow_summary['arrowRT_sem_smooth'] = smooth_series(arrow_summary['arrowRT_sem'], window=10)

# -------------------------------------------------
# Plot (1x2 subplots)
fig, axes = plt.subplots(1, 2, figsize=(6, 3), dpi=300, sharey=True)

# --- SpaceRT subplot
ax = axes[0]
ax.plot(space_summary['trial'], space_summary['spaceRT_mean_smooth'],
        color='#ff7f0e', label='spaceRT', linewidth=1)
ax.fill_between(space_summary['trial'],
                space_summary['spaceRT_mean_smooth'] - space_summary['spaceRT_sem_smooth'],
                space_summary['spaceRT_mean_smooth'] + space_summary['spaceRT_sem_smooth'],
                color='#ff7f0e', alpha=0.2)
ax.set_title("flip RT", fontsize=13, fontweight='bold')
ax.set_xlabel("trial", fontsize=12, fontweight='bold')
ax.set_ylabel("RT (ms)", fontsize=12, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.set_major_locator(MaxNLocator(nbins=6, integer=False))

# --- ArrowRT subplot
ax = axes[1]
ax.plot(arrow_summary['trial'], arrow_summary['arrowRT_mean_smooth'],
        color='#2ca02c', label='arrowRT', linewidth=1)
ax.fill_between(arrow_summary['trial'],
                arrow_summary['arrowRT_mean_smooth'] - arrow_summary['arrowRT_sem_smooth'],
                arrow_summary['arrowRT_mean_smooth'] + arrow_summary['arrowRT_sem_smooth'],
                color='#2ca02c', alpha=0.2)
ax.set_title("choice RT", fontsize=13, fontweight='bold')
ax.set_xlabel("trial", fontsize=12, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.set_major_locator(MaxNLocator(nbins=6, integer=False))

plt.tight_layout()
os.makedirs(output_folder, exist_ok=True)
plt.savefig(os.path.join(output_folder, "trialwise_space_arrow_RT.pdf"),
            format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "trialwise_space_arrow_RT.svg"),
            format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_58680\951300538.py:105: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


<h2 style="color: green;">(2_2) acc for different numbers</h2>

# use scatter for this

In [ ]:
# Define color scheme
distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# --- Function to compute combined accuracy ---
def compute_combined_accuracy(dataframes):
    acc_by_card_block = []
    for df in dataframes:
        df['outcome'] = df['outcome'].astype(str)
        df['is_win'] = df['outcome'] == 'win'
        df['myCard'] = df['myCard'].astype(int)
        acc_means = df.groupby(['myCard', 'distribution', 'block_type'])['is_win'].mean().reset_index()
        acc_by_card_block.append(acc_means)
    return pd.concat(acc_by_card_block)

# --- Combine healthy and epileptic data, only 'fix' trials ---
combined_acc_all = compute_combined_accuracy(dataframes + dataframes_epileptic)
combined_acc_all = combined_acc_all[combined_acc_all['block_type'] == 'fix']

# --- Plot ---
fig, axes = plt.subplots(1,3, figsize=(6, 2.5), dpi=130, sharex=True)
fig.suptitle('accuracy across cards', fontsize=14)

for i, dist in enumerate(distributions):
    ax = axes[i]
    data = combined_acc_all[(combined_acc_all['distribution'] == dist)]
    # Boxplot without fill color, thinner lines
    sns.boxplot(
        data=data,
        x='myCard',
        y='is_win',
        ax=ax,
        showcaps=True,
        boxprops=dict(facecolor='none', edgecolor='black', linewidth=0.3),
        medianprops=dict(color='black', linewidth=0.3),
        whiskerprops=dict(color='black', linewidth=0.3),
        capprops=dict(color='black', linewidth=0.3),
        flierprops=dict(marker='')  # hide outliers
    )




    # Add scatter points
    for x in sorted(data['myCard'].unique()):
        y_vals = data[data['myCard'] == x]['is_win']
        jittered_x = np.random.normal(loc=x, scale=0.05, size=len(y_vals))
        jittered_y = y_vals + np.random.normal(loc=0, scale=0.04, size=len(y_vals))
        ax.scatter(jittered_x - 1, jittered_y, color=colors[dist], alpha=0.6, s=4, edgecolors='none')


    ax.set_title(dist, fontsize=12)
    ax.set_ylabel("accuracy", fontsize=11)
    ax.set_xlabel("myCard", fontsize=11)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim(0, 1.02)
    ax.set_xticks(range(0, 9))  # explicitly set 1–9
    ax.set_xticklabels([str(i) for i in range(1, 10)])

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig(os.path.join(output_folder, "accuracy_combined_fix_only.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "accuracy_combined_fix_only.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_58680\4250435869.py:64: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


<h2 style="color: green;">(3) Arrow Press</h2>

In [ ]:


distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# --- Sigmoid function ---
def sigmoid(x, L ,x0, k, b):
    return L / (1 + np.exp(-k*(x - x0))) + b

# --- Compute percentages ---
def compute_percentages(dataframes, target_dist):
    group_results = []

    for df in dataframes:
        sub_df = df[(df['distribution'] == target_dist) & (df['block_type'] == 'fix')].copy()
        sub_df['group'] = sub_df['myCard'].astype(str)

        group_counts = sub_df.groupby(['group', 'choice'], observed=True)['choice'].count().unstack(fill_value=0)
        group_totals = group_counts.sum(axis=1)
        group_percentages = (group_counts.T / group_totals).T * 100
        group_percentages['group'] = group_percentages.index
        group_results.append(group_percentages.reset_index(drop=True))

    if not group_results:
        return None, None, None

    combined_df = pd.concat(group_results)
    mean_df = combined_df.groupby('group', observed=True).mean().reset_index()
    sem_df = combined_df.groupby('group', observed=True).sem().reset_index()
    
    return mean_df, sem_df, mean_df['group']

# --- Combine healthy + epileptic ---
all_dataframes = dataframes + dataframes_epileptic

# --- Plot ---
fig, ax = plt.subplots(figsize=(3, 3), dpi=130)
# fig.suptitle('percentage of arrowup', fontsize=14)

for dist in distributions:
    mean_df, sem_df, groups = compute_percentages(all_dataframes, dist)
    if mean_df is None:
        continue

    arrow_up = mean_df.get('arrowup', 0).values
    sem_up = sem_df.get('arrowup', 0).values
    x = np.arange(len(groups))
    color = colors[dist]

    # Scatter with error bars
    ax.errorbar(
    x, arrow_up, yerr=sem_up,
    fmt='o',
    color=color,
    capsize=0.8,
    capthick=0.2,   # <- thinner cap lines
    markersize=0.8,
    elinewidth=0.2,
    label=dist
)



    # Sigmoid fit
    try:
        p0 = [max(arrow_up), np.median(x), 1, min(arrow_up)]  # initial guess
        popt, _ = curve_fit(sigmoid, x, arrow_up, p0, maxfev=10000)
        x_smooth = np.linspace(x.min(), x.max(), 200)
        y_smooth = sigmoid(x_smooth, *popt)
        ax.plot(x_smooth, y_smooth, color=color, linewidth=0.3, alpha=0.5, linestyle='--')

    except Exception as e:
        print(f"Sigmoid fit failed for {dist}: {e}")

ax.set_xticks(x)
ax.set_xticklabels(groups)
ax.set_ylim(0, 100)
ax.set_xlabel("Card", fontsize=11)
ax.set_ylabel("arrow up %", fontsize=11)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)



plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig(os.path.join(output_folder, "arrow_up_percent_fix_combined.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "arrow_up_percent_fix_combined.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_58680\4174515948.py:86: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


# checking if fix mix curves are significantly different

In [ ]:


distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# --- Sigmoid function ---
def sigmoid(x, L, x0, k, b):
    return L / (1 + np.exp(-k*(x - x0))) + b

# --- Compute percentages ---
def compute_percentages(dataframes, target_dist, block_type):
    group_results = []

    for df in dataframes:
        sub_df = df[(df['distribution'] == target_dist) & (df['block_type'] == block_type)].copy()
        sub_df['group'] = sub_df['myCard'].astype(str)

        group_counts = sub_df.groupby(['group', 'choice'], observed=True)['choice'].count().unstack(fill_value=0)
        group_totals = group_counts.sum(axis=1)
        group_percentages = (group_counts.T / group_totals).T * 100
        group_percentages['group'] = group_percentages.index
        group_results.append(group_percentages.reset_index(drop=True))

    if not group_results:
        return None, None, None

    combined_df = pd.concat(group_results)
    mean_df = combined_df.groupby('group', observed=True).mean().reset_index()
    sem_df = combined_df.groupby('group', observed=True).sem().reset_index()
    
    return mean_df, sem_df, mean_df['group']

# --- Plotting function ---
def plot_block(ax, dataframes, block_type):
    for dist in distributions:
        mean_df, sem_df, groups = compute_percentages(dataframes, dist, block_type)
        if mean_df is None:
            continue

        arrow_up = mean_df.get('arrowup', 0).values
        sem_up = sem_df.get('arrowup', 0).values
        x = np.arange(len(groups))
        color = colors[dist]

        ax.errorbar(
            x, arrow_up, yerr=sem_up,
            fmt='o',
            color=color,
            capsize=0.8,
            capthick=0.2,
            markersize=0.8,
            elinewidth=0.2,
            label=dist
        )

        # Sigmoid fit
        try:
            p0 = [max(arrow_up), np.median(x), 1, min(arrow_up)]
            popt, _ = curve_fit(sigmoid, x, arrow_up, p0, maxfev=10000)
            x_smooth = np.linspace(x.min(), x.max(), 200)
            y_smooth = sigmoid(x_smooth, *popt)
            ax.plot(x_smooth, y_smooth, color=color, linewidth=0.3, alpha=0.5, linestyle='--')
        except Exception as e:
            print(f"Sigmoid fit failed for {dist} in {block_type}: {e}")

    ax.set_xticks(x)
    ax.set_xticklabels(groups)
    ax.set_ylim(0, 100)
    ax.set_xlabel("Card", fontsize=11)
    ax.set_ylabel("arrow up %", fontsize=11)
    ax.set_title(f"{block_type}", fontsize=12, fontweight='bold')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# --- Combine healthy + epileptic ---
all_dataframes = dataframes + dataframes_epileptic

# --- Plot both blocks ---
fig, axes = plt.subplots(1, 2, figsize=(6.5, 3), dpi=130, sharey=True)

plot_block(axes[0], all_dataframes, block_type='fix')
plot_block(axes[1], all_dataframes, block_type='mix')

# axes[1].legend(title='Distribution', loc='lower right', fontsize=7, title_fontsize=8)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig(os.path.join(output_folder, "arrow_up_percent_fix_mix_combined.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "arrow_up_percent_fix_mix_combined.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_58680\1884144657.py:87: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:


distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# --- Sigmoid function ---
def sigmoid(x, L, x0, k, b):
    return L / (1 + np.exp(-k*(x - x0))) + b

# --- Compute percentages ---
def compute_percentages(dataframes, target_dist, block_type):
    group_results = []

    for df in dataframes:
        sub_df = df[(df['distribution'] == target_dist) & (df['block_type'] == block_type)].copy()
        sub_df['group'] = sub_df['myCard'].astype(str)

        group_counts = sub_df.groupby(['group', 'choice'], observed=True)['choice'].count().unstack(fill_value=0)
        group_totals = group_counts.sum(axis=1)
        group_percentages = (group_counts.T / group_totals).T * 100
        group_percentages['group'] = group_percentages.index
        group_results.append(group_percentages.reset_index(drop=True))

    if not group_results:
        return None, None, None

    combined_df = pd.concat(group_results)
    mean_df = combined_df.groupby('group', observed=True).mean().reset_index()
    sem_df = combined_df.groupby('group', observed=True).sem().reset_index()
    
    return mean_df, sem_df, mean_df['group']

# --- Plotting function ---
def plot_block(ax, dataframes, block_type):
    for dist in distributions:
        mean_df, sem_df, groups = compute_percentages(dataframes, dist, block_type)
        if mean_df is None:
            continue

        arrow_up = mean_df.get('arrowup', 0).values
        sem_up = sem_df.get('arrowup', 0).values
        x = np.arange(len(groups))
        color = colors[dist]

        ax.errorbar(
            x, arrow_up, yerr=sem_up,
            fmt='o',
            color=color,
            capsize=0.8,
            capthick=0.2,
            markersize=0.8,
            elinewidth=0.2,
            label=dist
        )

        # Sigmoid fit
        try:
            p0 = [max(arrow_up), np.median(x), 1, min(arrow_up)]
            popt, _ = curve_fit(sigmoid, x, arrow_up, p0, maxfev=10000)
            x_smooth = np.linspace(x.min(), x.max(), 200)
            y_smooth = sigmoid(x_smooth, *popt)
            ax.plot(x_smooth, y_smooth, color=color, linewidth=0.3, alpha=0.5, linestyle='--')
        except Exception as e:
            print(f"Sigmoid fit failed for {dist} in {block_type}: {e}")

    ax.set_xticks(x)
    ax.set_xticklabels(groups)
    ax.set_ylim(0, 100)
    ax.set_xlabel("Card", fontsize=11)
    ax.set_ylabel("arrow up %", fontsize=11)
    ax.set_title(f"{block_type} block", fontsize=12, fontweight='bold')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# --- Combine healthy + epileptic ---
all_dataframes = dataframes 

# --- Plot both blocks ---
fig, axes = plt.subplots(1, 2, figsize=(6.5, 3), dpi=130, sharey=True)

plot_block(axes[0], all_dataframes, block_type='fix')
plot_block(axes[1], all_dataframes, block_type='mix')

axes[1].legend(title='Distribution', loc='lower right', fontsize=7, title_fontsize=8)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig(os.path.join(output_folder, "arrow_up_percent_fix_mix_healthy.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "arrow_up_percent_fix_mix_healthy.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_58680\2368059135.py:87: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:


distributions = ['uniform', 'low', 'high']
colors = {'uniform': '#808080', 'low': '#ff7f0e', 'high': '#2ca02c'}

# --- Sigmoid function ---
def sigmoid(x, L, x0, k, b):
    return L / (1 + np.exp(-k*(x - x0))) + b

# --- Compute percentages ---
def compute_percentages(dataframes, target_dist, block_type):
    group_results = []

    for df in dataframes:
        sub_df = df[(df['distribution'] == target_dist) & (df['block_type'] == block_type)].copy()
        sub_df['group'] = sub_df['myCard'].astype(str)

        group_counts = sub_df.groupby(['group', 'choice'], observed=True)['choice'].count().unstack(fill_value=0)
        group_totals = group_counts.sum(axis=1)
        group_percentages = (group_counts.T / group_totals).T * 100
        group_percentages['group'] = group_percentages.index
        group_results.append(group_percentages.reset_index(drop=True))

    if not group_results:
        return None, None, None

    combined_df = pd.concat(group_results)
    mean_df = combined_df.groupby('group', observed=True).mean().reset_index()
    sem_df = combined_df.groupby('group', observed=True).sem().reset_index()
    
    return mean_df, sem_df, mean_df['group']

# --- Plotting function ---
def plot_block(ax, dataframes, block_type):
    for dist in distributions:
        mean_df, sem_df, groups = compute_percentages(dataframes, dist, block_type)
        if mean_df is None:
            continue

        arrow_up = mean_df.get('arrowup', 0).values
        sem_up = sem_df.get('arrowup', 0).values
        x = np.arange(len(groups))
        color = colors[dist]

        ax.errorbar(
            x, arrow_up, yerr=sem_up,
            fmt='o',
            color=color,
            capsize=0.8,
            capthick=0.2,
            markersize=0.8,
            elinewidth=0.2,
            label=dist
        )

        # Sigmoid fit
        try:
            p0 = [max(arrow_up), np.median(x), 1, min(arrow_up)]
            popt, _ = curve_fit(sigmoid, x, arrow_up, p0, maxfev=10000)
            x_smooth = np.linspace(x.min(), x.max(), 200)
            y_smooth = sigmoid(x_smooth, *popt)
            ax.plot(x_smooth, y_smooth, color=color, linewidth=0.3, alpha=0.5, linestyle='--')
        except Exception as e:
            print(f"Sigmoid fit failed for {dist} in {block_type}: {e}")

    ax.set_xticks(x)
    ax.set_xticklabels(groups)
    ax.set_ylim(0, 100)
    ax.set_xlabel("Card", fontsize=11)
    ax.set_ylabel("arrow up %", fontsize=11)
    ax.set_title(f"{block_type} block", fontsize=12, fontweight='bold')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# --- Combine healthy + epileptic ---
all_dataframes = dataframes_epileptic

# --- Plot both blocks ---
fig, axes = plt.subplots(1, 2, figsize=(6.5, 3), dpi=130, sharey=True)

plot_block(axes[0], all_dataframes, block_type='fix')
plot_block(axes[1], all_dataframes, block_type='mix')

axes[1].legend(title='Distribution', loc='lower right', fontsize=7, title_fontsize=8)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig(os.path.join(output_folder, "arrow_up_percent_fix_mix_epileptic.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "arrow_up_percent_fix_mix_epileptic.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_58680\616953441.py:87: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


# accuracy vs spaceRT and arrowRT

In [ ]:

# fix_accuracy = []
# fix_spaceRT = []
# mix_accuracy = []
# mix_spaceRT = []

# fix_arrowRT = []
# mix_arrowRT = []

# def remove_spaceRT_outliers(df):
#     q1 = df['spaceRT'].quantile(0.25)
#     q3 = df['spaceRT'].quantile(0.75)
#     iqr = q3 - q1
#     lower_bound = q1 - 1.5 * iqr
#     upper_bound = 1500
#     return df[(df['spaceRT'] >= lower_bound) & (df['spaceRT'] <= upper_bound)]





# for df in dataframes_epileptic:
#     df['is_win'] = df['outcome'].apply(lambda x: 1 if x == 'win' else 0)
    
#     fix_data = df[df['block_type'] == 'fix']
#     mix_data = df[df['block_type'] == 'mix']

#     # Remove outliers
#     fix_data = remove_spaceRT_outliers(fix_data)
#     mix_data = remove_spaceRT_outliers(mix_data)

#     fix_accuracy.append(fix_data['is_win'].mean())
#     fix_spaceRT.append(fix_data['spaceRT'].mean())
#     fix_arrowRT.append(fix_data['arrowRT'].mean())

#     mix_accuracy.append(mix_data['is_win'].mean())
#     mix_spaceRT.append(mix_data['spaceRT'].mean())
#     mix_arrowRT.append(mix_data['arrowRT'].mean())

# # --- Linear Regression and R² ---
# fix_spaceRT_arr = np.array(fix_spaceRT).reshape(-1, 1)
# fix_arrowRT_arr = np.array(fix_arrowRT).reshape(-1, 1)
# mix_spaceRT_arr = np.array(mix_spaceRT).reshape(-1, 1)
# mix_arrowRT_arr = np.array(mix_arrowRT).reshape(-1, 1)
# fix_accuracy_arr = np.array(fix_accuracy)
# mix_accuracy_arr = np.array(mix_accuracy)
# # --- Compute Pearson correlations ---
# corr_fix_space, _ = pearsonr(fix_spaceRT, fix_accuracy)
# corr_mix_space, _ = pearsonr(mix_spaceRT, mix_accuracy)
# corr_fix_arrow, _ = pearsonr(fix_arrowRT, fix_accuracy)
# corr_mix_arrow, _ = pearsonr(mix_arrowRT, mix_accuracy)

# # --- Create scatterplots ---
# fig, axes = plt.subplots(2, 2, figsize=(8, 8), dpi=1200, sharey='row')

# # Fix block: spaceRT
# axes[0, 0].scatter(fix_spaceRT, fix_accuracy, color='blue', alpha=0.6, edgecolors='none')
# axes[0, 0].set_title('fix', fontsize=12, fontweight='bold')
# axes[0, 0].set_xlabel('space RT (ms)', fontsize=11, fontweight='bold')
# axes[0, 0].set_ylabel('accuracy', fontsize=11, fontweight='bold')
# axes[0, 0].spines['top'].set_visible(False)
# axes[0, 0].spines['right'].set_visible(False)
# axes[0, 0].set_ylim(0.5, 1.02)
# axes[0, 0].text(0.05, 0.05, f"r = {corr_fix_space:.2f}", transform=axes[0, 0].transAxes, fontsize=10)

# # Mix block: spaceRT
# axes[0, 1].scatter(mix_spaceRT, mix_accuracy, color='orange', alpha=0.6, edgecolors='none')
# axes[0, 1].set_title('mix', fontsize=12, fontweight='bold')
# axes[0, 1].set_xlabel('space RT (ms)', fontsize=11, fontweight='bold')
# axes[0, 1].spines['top'].set_visible(False)
# axes[0, 1].spines['right'].set_visible(False)
# axes[0, 1].text(0.05, 0.05, f"r = {corr_mix_space:.2f}", transform=axes[0, 1].transAxes, fontsize=10)

# # Fix block: arrowRT
# axes[1, 0].scatter(fix_arrowRT, fix_accuracy, color='blue', alpha=0.6, edgecolors='none')
# axes[1, 0].set_xlabel('arrow RT (ms)', fontsize=11, fontweight='bold')
# axes[1, 0].set_ylabel('accuracy', fontsize=11, fontweight='bold')
# axes[1, 0].spines['top'].set_visible(False)
# axes[1, 0].spines['right'].set_visible(False)
# axes[1, 0].text(0.05, 0.05, f"r = {corr_fix_arrow:.2f}", transform=axes[1, 0].transAxes, fontsize=10)

# # Mix block: arrowRT
# axes[1, 1].scatter(mix_arrowRT, mix_accuracy, color='orange', alpha=0.6, edgecolors='none')
# axes[1, 1].set_xlabel('arrow RT (ms)', fontsize=11, fontweight='bold')
# axes[1, 1].spines['top'].set_visible(False)
# axes[1, 1].spines['right'].set_visible(False)
# axes[1, 1].text(0.05, 0.05, f"r = {corr_mix_arrow:.2f}", transform=axes[1, 1].transAxes, fontsize=10)

# plt.tight_layout()
# plt.savefig(os.path.join(output_folder, "accuracy_vs_spaceRT_arrowRT_fix_mix_epileptic.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
# plt.savefig(os.path.join(output_folder, "accuracy_vs_spaceRT_arrowRT_fix_mix_epileptic.svg"), format="svg", dpi=1200, bbox_inches="tight")
# plt.show()


In [ ]:

# fix_accuracy = []
# fix_spaceRT = []
# mix_accuracy = []
# mix_spaceRT = []

# fix_arrowRT = []
# mix_arrowRT = []

# def remove_spaceRT_outliers(df):
#     q1 = df['spaceRT'].quantile(0.25)
#     q3 = df['spaceRT'].quantile(0.75)
#     iqr = q3 - q1
#     lower_bound = q1 - 1.5 * iqr
#     upper_bound = 1500
#     return df[(df['spaceRT'] >= lower_bound) & (df['spaceRT'] <= upper_bound)]





# for df in dataframes:
#     df['is_win'] = df['outcome'].apply(lambda x: 1 if x == 'win' else 0)
    
#     fix_data = df[df['block_type'] == 'fix']
#     mix_data = df[df['block_type'] == 'mix']

#     # Remove outliers
#     fix_data = remove_spaceRT_outliers(fix_data)
#     mix_data = remove_spaceRT_outliers(mix_data)

#     fix_accuracy.append(fix_data['is_win'].mean())
#     fix_spaceRT.append(fix_data['spaceRT'].mean())
#     fix_arrowRT.append(fix_data['arrowRT'].mean())

#     mix_accuracy.append(mix_data['is_win'].mean())
#     mix_spaceRT.append(mix_data['spaceRT'].mean())
#     mix_arrowRT.append(mix_data['arrowRT'].mean())

# # --- Linear Regression and R² ---
# fix_spaceRT_arr = np.array(fix_spaceRT).reshape(-1, 1)
# fix_arrowRT_arr = np.array(fix_arrowRT).reshape(-1, 1)
# mix_spaceRT_arr = np.array(mix_spaceRT).reshape(-1, 1)
# mix_arrowRT_arr = np.array(mix_arrowRT).reshape(-1, 1)
# fix_accuracy_arr = np.array(fix_accuracy)
# mix_accuracy_arr = np.array(mix_accuracy)
# # --- Compute Pearson correlations ---
# corr_fix_space, _ = pearsonr(fix_spaceRT, fix_accuracy)
# corr_mix_space, _ = pearsonr(mix_spaceRT, mix_accuracy)
# corr_fix_arrow, _ = pearsonr(fix_arrowRT, fix_accuracy)
# corr_mix_arrow, _ = pearsonr(mix_arrowRT, mix_accuracy)

# # --- Create scatterplots ---
# fig, axes = plt.subplots(2, 2, figsize=(8, 8), dpi=1200, sharey='row')

# # Fix block: spaceRT
# axes[0, 0].scatter(fix_spaceRT, fix_accuracy, color='blue', alpha=0.6, edgecolors='none')
# axes[0, 0].set_title('fix', fontsize=12, fontweight='bold')
# axes[0, 0].set_xlabel('space RT (ms)', fontsize=11, fontweight='bold')
# axes[0, 0].set_ylabel('accuracy', fontsize=11, fontweight='bold')
# axes[0, 0].spines['top'].set_visible(False)
# axes[0, 0].spines['right'].set_visible(False)
# axes[0, 0].set_ylim(0.5, 1.02)
# axes[0, 0].text(0.05, 0.05, f"r = {corr_fix_space:.2f}", transform=axes[0, 0].transAxes, fontsize=10)

# # Mix block: spaceRT
# axes[0, 1].scatter(mix_spaceRT, mix_accuracy, color='orange', alpha=0.6, edgecolors='none')
# axes[0, 1].set_title('mix', fontsize=12, fontweight='bold')
# axes[0, 1].set_xlabel('space RT (ms)', fontsize=11, fontweight='bold')
# axes[0, 1].spines['top'].set_visible(False)
# axes[0, 1].spines['right'].set_visible(False)
# axes[0, 1].text(0.05, 0.05, f"r = {corr_mix_space:.2f}", transform=axes[0, 1].transAxes, fontsize=10)

# # Fix block: arrowRT
# axes[1, 0].scatter(fix_arrowRT, fix_accuracy, color='blue', alpha=0.6, edgecolors='none')
# axes[1, 0].set_xlabel('arrow RT (ms)', fontsize=11, fontweight='bold')
# axes[1, 0].set_ylabel('accuracy', fontsize=11, fontweight='bold')
# axes[1, 0].spines['top'].set_visible(False)
# axes[1, 0].spines['right'].set_visible(False)
# axes[1, 0].text(0.05, 0.05, f"r = {corr_fix_arrow:.2f}", transform=axes[1, 0].transAxes, fontsize=10)

# # Mix block: arrowRT
# axes[1, 1].scatter(mix_arrowRT, mix_accuracy, color='orange', alpha=0.6, edgecolors='none')
# axes[1, 1].set_xlabel('arrow RT (ms)', fontsize=11, fontweight='bold')
# axes[1, 1].spines['top'].set_visible(False)
# axes[1, 1].spines['right'].set_visible(False)
# axes[1, 1].text(0.05, 0.05, f"r = {corr_mix_arrow:.2f}", transform=axes[1, 1].transAxes, fontsize=10)

# plt.tight_layout()
# plt.savefig(os.path.join(output_folder, "accuracy_vs_spaceRT_arrowRT_fix_mix_healthy.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
# plt.savefig(os.path.join(output_folder, "accuracy_vs_spaceRT_arrowRT_fix_mix_healthy.svg"), format="svg", dpi=1200, bbox_inches="tight")
# plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import os

fix_accuracy = []
fix_spaceRT = []
mix_accuracy = []
mix_spaceRT = []

fix_arrowRT = []
mix_arrowRT = []

# --- Outlier removal for spaceRT ---
def remove_spaceRT_outliers(df):
    q1 = pd.to_numeric(df['spaceRT'], errors='coerce').quantile(0.25)
    q3 = pd.to_numeric(df['spaceRT'], errors='coerce').quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = 2000
    df['spaceRT'] = pd.to_numeric(df['spaceRT'], errors='coerce')
    return df[(df['spaceRT'] >= lower_bound) & (df['spaceRT'] <= upper_bound)]

# --- Permutation test for correlation ---
def permutation_pval(x, y, n_perms=10000, seed=42):
    rng = np.random.default_rng(seed)
    observed_r, _ = pearsonr(x, y)
    null_rs = []
    for _ in range(n_perms):
        y_perm = rng.permutation(y)
        r_perm, _ = pearsonr(x, y_perm)
        null_rs.append(r_perm)
    p_val = np.mean(np.abs(null_rs) >= abs(observed_r))
    return observed_r, p_val

# --- Process data ---
for df in dataframes + dataframes_epileptic:
    df['is_win'] = df['outcome'].apply(lambda x: 1 if x == 'win' else 0)

    # force numeric for both RT columns
    df['spaceRT'] = pd.to_numeric(df['spaceRT'], errors='coerce')
    df['arrowRT'] = pd.to_numeric(df['arrowRT'], errors='coerce')

    fix_data = df[df['block_type'] == 'fix'].copy()
    mix_data = df[df['block_type'] == 'mix'].copy()

    # Remove outliers
    fix_data = remove_spaceRT_outliers(fix_data)
    mix_data = remove_spaceRT_outliers(mix_data)

    # Drop any rows where RTs are NaN
    fix_data = fix_data.dropna(subset=['spaceRT', 'arrowRT'])
    mix_data = mix_data.dropna(subset=['spaceRT', 'arrowRT'])

    # Collect averages
    fix_accuracy.append(fix_data['is_win'].mean())
    fix_spaceRT.append(fix_data['spaceRT'].mean())
    fix_arrowRT.append(fix_data['arrowRT'].mean())

    mix_accuracy.append(mix_data['is_win'].mean())
    mix_spaceRT.append(mix_data['spaceRT'].mean())
    mix_arrowRT.append(mix_data['arrowRT'].mean())

# --- Arrays for regression ---
fix_spaceRT_arr = np.array(fix_spaceRT).reshape(-1, 1)
fix_arrowRT_arr = np.array(fix_arrowRT).reshape(-1, 1)
mix_spaceRT_arr = np.array(mix_spaceRT).reshape(-1, 1)
mix_arrowRT_arr = np.array(mix_arrowRT).reshape(-1, 1)
fix_accuracy_arr = np.array(fix_accuracy)
mix_accuracy_arr = np.array(mix_accuracy)

# --- Correlations ---
corr_fix_space, p_fix_space = permutation_pval(fix_spaceRT, fix_accuracy)
corr_mix_space, p_mix_space = permutation_pval(mix_spaceRT, mix_accuracy)
corr_fix_arrow, p_fix_arrow = permutation_pval(fix_arrowRT, fix_accuracy)
corr_mix_arrow, p_mix_arrow = permutation_pval(mix_arrowRT, mix_accuracy)

spaceRT_max = max(max(fix_spaceRT), max(mix_spaceRT))
arrowRT_max = max(max(fix_arrowRT), max(mix_arrowRT))

# --- Plotting ---
fig, axes = plt.subplots(2, 2, figsize=(4, 4), dpi=1200, sharey='row')
scatter_dots = 8

# Fix block: spaceRT
axes[0, 0].scatter(fix_spaceRT, fix_accuracy, color='pink', s=scatter_dots, alpha=1, edgecolors='none')
fit = np.polyfit(fix_spaceRT, fix_accuracy, 1)
axes[0, 0].plot(np.sort(fix_spaceRT), np.poly1d(fit)(np.sort(fix_spaceRT)), linestyle='--', color='black', linewidth=1)
axes[0, 0].set_title('fix', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('space RT (ms)', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('accuracy', fontsize=11, fontweight='bold')
axes[0, 0].spines['top'].set_visible(False)
axes[0, 0].spines['right'].set_visible(False)
axes[0, 0].set_ylim(0.5, 1.0)
axes[0, 0].set_xlim(0, spaceRT_max * 1.02)
axes[0, 0].text(0.05, 0.05, f"r = {corr_fix_space:.2f}, p = {p_fix_space:.3f}", transform=axes[0, 0].transAxes, fontsize=8)

# Mix block: spaceRT
axes[0, 1].scatter(mix_spaceRT, mix_accuracy, color='pink', s=scatter_dots, alpha=1, edgecolors='none')
fit = np.polyfit(mix_spaceRT, mix_accuracy, 1)
axes[0, 1].plot(np.sort(mix_spaceRT), np.poly1d(fit)(np.sort(mix_spaceRT)), linestyle='--', color='black', linewidth=1)
axes[0, 1].set_title('mix', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('space RT (ms)', fontsize=11, fontweight='bold')
axes[0, 1].spines['top'].set_visible(False)
axes[0, 1].spines['right'].set_visible(False)
axes[0, 1].set_ylim(0.5, 1.0)
axes[0, 1].set_xlim(0, spaceRT_max * 1.02)
axes[0, 1].text(0.05, 0.05, f"r = {corr_mix_space:.2f}, p = {p_mix_space:.3f}", transform=axes[0, 1].transAxes, fontsize=8)

# Fix block: arrowRT
axes[1, 0].scatter(fix_arrowRT, fix_accuracy, color='pink', s=scatter_dots, alpha=1, edgecolors='none')
fit = np.polyfit(fix_arrowRT, fix_accuracy, 1)
axes[1, 0].plot(np.sort(fix_arrowRT), np.poly1d(fit)(np.sort(fix_arrowRT)), linestyle='--', color='black', linewidth=1)
axes[1, 0].set_xlabel('arrow RT (ms)', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('accuracy', fontsize=11, fontweight='bold')
axes[1, 0].spines['top'].set_visible(False)
axes[1, 0].spines['right'].set_visible(False)
axes[1, 0].set_ylim(0.5, 1.0)
axes[1, 0].set_xlim(0, arrowRT_max * 1.02)
axes[1, 0].text(0.05, 0.05, f"r = {corr_fix_arrow:.2f}, p = {p_fix_arrow:.3f}", transform=axes[1, 0].transAxes, fontsize=8)

# Mix block: arrowRT
axes[1, 1].scatter(mix_arrowRT, mix_accuracy, color='pink', s=scatter_dots, alpha=1, edgecolors='none')
fit = np.polyfit(mix_arrowRT, mix_accuracy, 1)
axes[1, 1].plot(np.sort(mix_arrowRT), np.poly1d(fit)(np.sort(mix_arrowRT)), linestyle='--', color='black', linewidth=1)
axes[1, 1].set_xlabel('arrow RT (ms)', fontsize=11, fontweight='bold')
axes[1, 1].spines['top'].set_visible(False)
axes[1, 1].spines['right'].set_visible(False)
axes[1, 1].set_ylim(0.5, 1.0)
axes[1, 1].set_xlim(0, arrowRT_max * 1.02)
axes[1, 1].text(0.05, 0.05, f"r = {corr_mix_arrow:.2f}, p = {p_mix_arrow:.3f}", transform=axes[1, 1].transAxes, fontsize=8)

plt.tight_layout()
plt.savefig(os.path.join(output_folder, "accuracy_vs_spaceRT_arrowRT_fix_mix_combined.pdf"), format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "accuracy_vs_spaceRT_arrowRT_fix_mix_combined.svg"), format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_58680\4171148312.py:137: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()
